In [1]:
import requests
import json
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime
import sys
sys.path.append('/Users/MikeTodisco/opt/anaconda3/lib/python3.7/site-packages')

from draft_kings import Client
from draft_kings.data import Sport
from tqdm.notebook import tqdm


In [2]:
response = requests.get('http://www.draftkings.com/lobby/getcontests?sport=GOLF?')

In [3]:
r = requests.get('https://datagolf.com/fantasy-projections')
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
from selenium import webdriver

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [7]:
site = 'https://datagolf.com/fantasy-projections'
wd = webdriver.Chrome('/Users/MikeTodisco/Documents/chromedriver', options=options, )
wd.get(site)

In [16]:
soup = BeautifulSoup(wd.page_source, 'lxml')

In [22]:
soup.find_all(class_='name-first')

[<span class="name-first">HIDEKI</span>,
 <span class="name-first">LOUIS</span>,
 <span class="name-first">WEBB</span>,
 <span class="name-first">WILL</span>,
 <span class="name-first">JASON</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <span class="name-first">PEEKING</span>,
 <s

In [23]:
soup.find_all(class_='dollar-signs')

[<a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="11000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="10700">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="10600">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="10300">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="10000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0.8)" value="6000">$</a>,
 <a class="dollar-signs" style="color: rgba(255, 133, 27,  0

In [24]:
soup.find_all(class_='scoring-pts')

[<a class="scoring-pts" id="80.25715_80.1731_-0.7230500000000011" value="80.5411">79.8</a>,
 <a class="scoring-pts" id="76.1541_88.29085_-1.8113499999999898" value="80.00675">78.2</a>,
 <a class="scoring-pts" id="80.79625_66.32075_6.01035" value="76.006">82.0</a>,
 <a class="scoring-pts" id="73.2319_71.305_-3.68795" value="72.6635">69.0</a>,
 <a class="scoring-pts" id="70.2485_73.92725_-0.8695999999999909" value="71.64115">70.8</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="scoring-pts" id="57.29795_51.79845_0.880899999999997" value="55.52575">56.4</a>,
 <a class="

In [8]:
html = wd.page_source

In [9]:
import pandas as pd

In [10]:
#!pip install html5lib

In [12]:
html

'<html color-mode="light"><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n\n\n     <!-- Global Site Tag (gtag.js) - Google Analytics -->\n\n     <script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-96115146-1"></script>\n     <script>\n       window.dataLayer = window.dataLayer || [];\n       function gtag(){dataLayer.push(arguments)};\n       gtag(\'js\', new Date());\n\n       gtag(\'config\', \'UA-96115146-1\');\n     </script>\n     <script>\n       if (\n         localStorage.getItem("color-mode") === "dark" ||\n         (window.matchMedia("(prefers-color-scheme: dark)").matches &&\n           !localStorage.getItem("color-mode"))\n       ) {\n         document.documentElement.setAttribute("color-mode", "dark");\n       }\n     </script>\n\n\n   <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/boots

In [ ]:
soup.find_all(class_="row projection-table")

In [ ]:
[t.get_text() for t in soup.select("value_params")]

In [ ]:
json.loads(soup.text)

In [ ]:
print(soup.prettify())

In [ ]:
contests = Client().contests(sport=Sport.NBA)
#contests

In [ ]:
contests

In [ ]:


def get_draft_group_id(contest_obj, contest_type_id=84):
    draft_groups = contest_obj.draft_groups
    draft_group_id_lst = []
    for x in draft_groups:
        if x.contest_type_id == contest_type_id:
            draft_group_id_lst.append(x.draft_group_id)
            
    return min(draft_group_id_lst)

draft_group_id = get_draft_group_id(contests)

In [ ]:
def get_draftables(draft_group_id):
    df_lst = []
    player_lst = Client().draftables(draft_group_id=draft_group_id).players
    for player in tqdm(player_lst):
        name_details = player.name_details
        first = name_details.first
        last = name_details.last
        name = f'{last}, {first}'
        salary = player.salary

        df = pd.DataFrame({
            'player_name': name,
            'showdown_salary': salary
        }, index=[0])

        df_lst.append(df)

    return pd.concat(df_lst).reset_index(drop=True)

In [ ]:
draftable_df = get_draftables(draft_group_id)
draftable_df.to_csv('draftables.csv', index=False)

In [ ]:
draft_group_id

In [ ]:
MY_API_KEY = 'dac62093607b097430db71962680'
API_PATH = 'https://feeds.datagolf.com/'
FILE_FORMAT = 'json'

In [ ]:
tour_lst = [
    'PGA',
    'OPP',
    'EUR',
    'KFT',
]

In [ ]:
config = {
    'field_updates':
    [
        'dg_id',
        'player_name',
        'country',
        'event_name',
        'current_round',
        'dk_id',
        'dk_salary',
        'fd_id',
        'fd_salary',
        'yh_id',
        'yh_salary',
        'early_late',
        'last_updated',
    ],
    'rankings':
    [
        'dg_id',
        #'player_name',
        'primary_tour',
        'datagolf_rank',
        'owgr_rank',
        'dg_skill_estimate'
    ],
    'pre_tourney_preds':
    [
        'dg_id',
        #'player_name',
        'make_cut_baseline',
        'top_10_baseline', 
        'top_20_baseline', 
        'top_5_baseline',
        'win_baseline', 
        'make_cut_baseline_history_fit',
        'top_10_baseline_history_fit', 
        'top_20_baseline_history_fit',
        'top_5_baseline_history_fit', 
        'win_baseline_history_fit'
    ],
    'skill_decomps':
    ['dg_id',
     #'player_name',
     'baseline_pred', 
     'timing_adjustment',
     'age_adjustment',
     #'cf_approach_comp',
     #'cf_short_comp',
     'strokes_gained_category_adjustment',
     'course_experience_adjustment',
     'course_history_adjustment', 
     'total_course_history_adjustment', 
     'total_fit_adjustment',
     'driving_accuracy_adjustment',
     'driving_distance_adjustment', 
     'other_fit_adjustment',
     'final_pred',
    ]
}

In [ ]:
def get_field_updates(path, tour, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&key={key}')
    data = response.json()
    df = pd.DataFrame(data['field'])
    
    for col in data.keys():
        if col != 'field':
            df[col] = data[col]
    
    return df[config['field_updates']]

def get_rankings(path, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?&key={key}')
    data = response.json()
    df = pd.DataFrame(data['rankings'])
    
    return df[config['rankings']]

def get_pre_tourney_preds(path, tour, odds_format='percent', key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&odds_format={odds_format}&key={key}')
    data = response.json()
    
    model_dict = {}
    for model in data['models_available']:
        df = pd.DataFrame(data[model])
        pred_cols = ['make_cut','top_10','top_20','top_5','win']
        df.rename(
            columns={x: x + f'_{model}' for x in pred_cols}, inplace=True
        )
        
        model_dict[model] = df
        
    cols_to_use = [col for col in model_dict['baseline_history_fit'].columns if 'history' in col] + ['dg_id']
    
    df = (model_dict['baseline']
          .merge(model_dict['baseline_history_fit'][cols_to_use],
                 how='left',
                 on='dg_id'
                )
         )
    
    return df[config['pre_tourney_preds']]

def get_player_skill_decomps(path, tour, key=MY_API_KEY):
    response = requests.get(f'{API_PATH}{path}?tour={tour}&key={key}')
    data = response.json()
    
    df = pd.DataFrame(data['players'])
    
    return df[config['skill_decomps']]

In [ ]:
def merge_dfs(dfs):
    merged_dfs = reduce(lambda left, right: pd.merge(left, right, on='dg_id', how='left'), dfs)
    
    return merged_dfs

In [ ]:
field_updates_df = get_field_updates('field-updates', 'pga')
rank_df = get_rankings('preds/get-dg-rankings')
pred_data = get_pre_tourney_preds('preds/pre-tournament', 'pga')
skill_df = get_player_skill_decomps('preds/player-decompositions?', 'pga')

In [ ]:
merged_df = merge_dfs([field_updates_df, rank_df, pred_data, skill_df])

In [ ]:
now = datetime.now()
merged_df.to_csv(f'data_golf_{now.strftime("%Y-%m-%d")}.csv', index=False)

In [ ]:
player_name_map = {
    'Lee, Kyoung-Hoon':'Lee, Kyounghoon',
    'Fitzpatrick, Matt':'Fitzpatrick, Matthew'
}

In [ ]:
draftable_df['player_name'] = (np.where(draftable_df['player_name'].isin(player_name_map.keys()),
                                        draftable_df['player_name'].map(player_name_map),
                                        draftable_df['player_name'])
                              )

In [ ]:
showdown_df = merged_df.merge(draftable_df, how = 'left', on ='player_name')